# Iris Lunar Rover — TVAC Testing Backend

Note: **Press the `green refresh button` then the `double play button` to reload and run.**

## Intro

This interface will allow you to issue commands and monitor the status of the rover. Functions you call here talk to the GSW backend and perform various analysis or debugging. Here's the set of tools available to you (note: they are also demoed below):

- `connect_serial(device_name)`: Initiate the connection with the rover over the RS-422 serial adapter.

- `send_command(command_name, **args)`: Send a command to the rover. `**args` means you enter the argument names and values in a list. For example: `send_command('WatchDogInterface_SetVSetpoint', setpoint=1000)`. Make sure to get the spelling of the command name *and* arg names correct. Note: some commands don't take arguments.

- `list_commands()`: List all commands you can send to the rover.

- `list_channels()`: List all telemetry channels which are being logged, both real and generated.

- `stream_data()`: Actively monitor and log data coming in from the rover. This should be your primary state.

- `plot_stream(channel_name)` Display a plot of all the data which has been logged for the channel with the given name.

<br/>

*Note:* If you want to prevent one of these functions from executing, you can comment it out by putting a `#` in front of it or pressing `ctrl + /` to toggle.

**Note:** You'll want to run `pyenv exec python tvac_telem.py` from this directory in a separate terminal window in order to collect telemetry during the test.

<br />

### Convenience functions:
The following functions exist to convert units. This is particularly for changing the heater settings. The heater takes it setpoint as an `adc` target and it returns its current state as an `adc` value as well.
- `degC_to_adc(temp)`: Converts a temperature in °Celcius to an `adc` target.
- `degK_to_adc(temp)`: Converts a temperature in °Kelvin to an `adc` target.
- `adc_to_degC(adc)`: Converts an `adc` reading into a temperature in °Celcius.
- `adc_to_degK(adc)`: Converts an `adc` reading into a temperature in °Kelvin.

**NOTE:** These conversions are only for °C and °K **not** C° or K°, which is to say, they're for absolute temperatures but not temperature differences - i.e. *dont* use these functions to set a window width for the heater.


## Setup:

In [1]:
from tvac_tools import *
connect_serial(device = '/dev/ttyUSB0')

Failed to connect to serial device. Is the device name right? Check the connection? Original error: [Errno 2] could not open port /dev/ttyUSB0: [Errno 2] No such file or directory: '/dev/ttyUSB0'


In [2]:
# settings['SAVE_FILE_PREFIX'] = 'iris_log' # this the prefix on all log files. make it something unique.
load_cache()

## Examples:

In [3]:
list_commands()
list_channels()


	0.	'WatchDogInterface_ResetSpecific', args: [reset_value: enum/*int32*/]

			Available values for `reset_value`:

				'NO_RESET' or 0 or 0x0

				'RESET_HERCULES' or 1 or 0x1

				'HERCULES_POWER_ON' or 2 or 0x2

				'HERCULES_POWER_OFF' or 3 or 0x3

				'RESET_RADIO' or 4 or 0x4

				'RADIO_POWER_ON' or 5 or 0x5

				'RADIO_POWER_OFF' or 6 or 0x6

				'RESET_FPGA' or 7 or 0x7

				'FPGA_POWER_ON' or 8 or 0x8

				'FPGA_POWER_OFF' or 9 or 0x9

				'RESET_MOTOR_1' or 10 or 0xa

				'RESET_MOTOR_2' or 11 or 0xb

				'RESET_MOTOR_3' or 12 or 0xc

				'RESET_MOTOR_4' or 13 or 0xd

				'RESET_ALL_MOTORS' or 14 or 0xe

				'ALL_MOTORS_ON' or 15 or 0xf

				'ALL_MOTORS_OFF' or 16 or 0x10

				'RESET_EN_3_3' or 17 or 0x11

				'EN_3_3_POWER_ON' or 18 or 0x12

				'EN_3_3_POWER_OFF' or 19 or 0x13

				'RESET_24_EN' or 20 or 0x14

				'EN_24_ON' or 21 or 0x15

				'EN_24_OFF' or 22 or 0x16

				'HDRM_OFF' or 24 or 0x18

				'FPGA_CAM_0' or 25 or 0x19

				'FPGA_CAM_1' or 26 or 0x1a

				'BATTE

In [4]:
send_command('WatchDogInterface_SwitchToKeepAliveMode', confirm='YES')

Can't send data, serial connection not started. Try `connect_serial()`.


'Data: 45 00 00 27 00 01 00 00 40 11 5e 27 de ad be ef 7f 00 00 01 00 35 1f 90 00 13 71 8c 00 07 00 00 ee ff 00 c0 eb 10 77.'

## Initial Setup Commands to Send:


In [5]:
send_command('WatchDogInterface_SwitchToKeepAliveMode', confirm='YES') # should already be in KeepAlive, just double checking

Can't send data, serial connection not started. Try `connect_serial()`.


'Data: 45 00 00 27 00 01 00 00 40 11 5e 27 de ad be ef 7f 00 00 01 00 35 1f 90 00 13 71 8c 00 07 00 00 ee ff 00 c0 eb 10 77.'

In [6]:
send_command('WatchDogInterface_ResetSpecific', reset_value='ENABLE_HEATER_CONTROL') # despite the name 'reset', this allows the heater to turn on

Can't send data, serial connection not started. Try `connect_serial()`.


'Data: 45 00 00 27 00 01 00 00 40 11 5e 27 de ad be ef 7f 00 00 01 00 35 1f 90 00 13 b4 8d 00 07 00 00 ee ff 00 c0 00 10 1f.'

In [7]:
send_command('WatchDogInterface_SetHeaterSetpoint', adc_setpoint=degC_to_adc(40))

Can't send data, serial connection not started. Try `connect_serial()`.


'Data: 45 00 00 28 00 01 00 00 40 11 5e 26 de ad be ef 7f 00 00 01 00 35 1f 90 00 14 88 85 00 08 00 00 ee ff 00 c0 da 10 71 04.'

In [8]:
send_command('WatchDogInterface_SetKpSpecific', value=0xBEEF) # default Kp is waay too small. change to something big to make it actually repsond

Can't send data, serial connection not started. Try `connect_serial()`.


'Data: 45 00 00 28 00 01 00 00 40 11 5e 26 de ad be ef 7f 00 00 01 00 35 1f 90 00 14 39 cb 00 08 00 00 ee ff 00 c0 aa 10 ef be.'

In [10]:
load_cache()
plot_stream('AdcTempKelvin')

SyntaxError: invalid syntax (<ipython-input-10-5e04e51d9746>, line 2)

In [9]:
stream_data()

Can't read data, serial connection not started. Try `connect_serial()`.
